# Sentinel-2 Access with STAC and AWS Open Data Registry

### AWS Open Data Registry
The AWS Open Data Registry hosts many different geospatial datasets. Sentinel-2, for example, is stored in a requester-pays S3 bucket which is made available for anyone to access and download or copy to their own S3 bucket. 

### STAC Specification

STAC stands for spatiotemporal asset catalogs. STAC is a specification for indexing geospatial datasets and aims to make data more accessible by standardizing the way that is searched for and discovered. 

Anyone interested in accessing data can use STAC to easily find relevant data and reduce the need to write custom code for each dataset integration. 

> "The goal is for all providers of spatiotemporal assets (Imagery, SAR, Point Clouds, Data Cubes, Full Motion Video, etc) to expose their data as SpatioTemporal Asset Catalogs (STAC), so that new code doesn't need to be written whenever a new data set or API is released." ([stacspec.org](https://stacspec.org/en/))

### Useful Links: 

- [Sentinel-2 Cloud-Optimized GeoTIFFs on AWS Registry of Open Data](https://registry.opendata.aws/sentinel-2-l2a-cogs/)
- [STAC Specification](https://stacspec.org/en)
- [AWS Open Data Registry GitHub Repo](https://github.com/awslabs/open-data-registry)
- [Carpentries Tutorial on Accessing Satellite Imagery with STAC](https://carpentries-incubator.github.io/geospatial-python/05-access-data.html)

This Jupyter Notebook walks through the process of searching for Sentinel-2 data matching a user's data request using STAC and then copying the relevant Sentinel-2 scenes from the source bucket on AWS Open Data Registry to the destination bucket in the Cecil science AWS account.  

### 

### Import required packages
We use the package `pystac_client` to read the url for the Earth Search STAC catalog API. The Earth Search catalog is a central search catalog for the open geospatial datasets on AWS, including Sentinel-2. 

We use `json` to construct metadata files for each Sentinel-2 scene.

In [7]:
from pystac_client import Client
import json

### Search for data matching our parameters.
Next, we set up the conditions we will search for. For simplicity, this example is using a bounding box representing the Kakadu National Park AOI. We also specify a start and end time, and the name of the collection we are searching for (in this case, "sentinel-2-l2a").

We then search the Earth Search STAC catalog using these parameters and examine the items that are returned. 

In [8]:
# Kakadu National Park as bounding box
bbox = [132.52934211276073, -12.730063400794094, 132.54027735328083, -12.721072673008706]
start_time = "2023-09-01"
end_time = "2023-09-20"
collection_name = "sentinel-2-l2a"


# STAC Catalog URL to search for scenes.
catalog = Client.open("https://earth-search.aws.element84.com/v1/")

# Search for items instersecting our bbox and within desired date range.
query = catalog.search(
    collections=[collection_name],
    datetime=f"{start_time}/{end_time}",
    bbox=bbox
)

# TODO: subset the scenes before copying them over

# Print the number of items found from executing the query.
items = query.item_collection()
print(f"Found {len(items)} Sentinel-2 scenes")

# Convert the query result to a json format (if desired to get full list of matching scenes and all metzdata).
stac_json = query.item_collection_as_dict()

# Print information about each item in the collection.
for item in items:
    print(item.id)
    print(item.datetime)
    print(item.properties)
    print(item.assets.keys())

Found 3 Sentinel-2 scenes
S2A_52LHM_20230918_0_L2A
2023-09-18 01:30:48.372000+00:00
{'created': '2023-09-18T07:42:46.728Z', 'platform': 'sentinel-2a', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 5.355347, 'mgrs:utm_zone': 52, 'mgrs:latitude_band': 'L', 'mgrs:grid_square': 'HM', 'grid:code': 'MGRS-52LHM', 'view:sun_azimuth': 60.28929408308, 'view:sun_elevation': 62.529655401736804, 's2:degraded_msi_data_percentage': 0.0017, 's2:nodata_pixel_percentage': 1.603352, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0.000455, 's2:cloud_shadow_percentage': 0.192144, 's2:vegetation_percentage': 17.785923, 's2:not_vegetated_percentage': 36.154309, 's2:water_percentage': 39.901057, 's2:unclassified_percentage': 0.610766, 's2:medium_proba_clouds_percentage': 0.641282, 's2:high_proba_clouds_percentage': 0.558218, 's2:thin_cirrus_percentage': 4.155846, 's2:snow_ice_percentage': 0, 's2:product_type': 'S2MSI2A', 's2:processing_baseline': '05.09

To keep track of all the metadata information associated with each of the assets, we can use the to_dict() function to convert the STAC response to a json file.

In [ ]:
import json
with open('test_metadata_json.json', 'w') as f:
    item_dict = item.to_dict()
    json.dump(item_dict, f, indent=2)

In [ ]:
# Open the json metadata
with open('test_metadata_json.json', 'r') as f:
    item_metadata = json.load(f)

# These are the bands that require scale factors applied to them.
bands_to_scale = ['aot', 'blue', 'coastal', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 
                     'rededge2', 'rededge3', 'swir16', 'swir22', 'wvp']

# These are the other bands that we want, but they do not require scale factors.
other_bands_of_interest = ['scl', 'visual'] 

# Now, we can get the scale factor that needs to be applied to each band like this:
for band in bands_to_scale:
    print(f'BAND: {band}')
    print(f'Scale factor: {item_metadata["assets"][band]["raster:bands"][0]["scale"]}')
    print(f'Offset: {item_metadata["assets"][band]["raster:bands"][0]["offset"]}') 
    print(f'Resolution: {item_metadata["assets"][band]["raster:bands"][0]["spatial_resolution"]}')
    print('--------------')


### Examine Asset Structure in AWS S3 Bucket
Each item (Sentinel-2 scene) has multiple assets (bands). We can examine the assets and the download links to each one in the following code block. This also indicates the structure of the data and how it is stored in the source S3 bucket.

In [ ]:
# Set some bands of interest to examine.
bands_of_interest = ['blue', 'green', 'red', 'granule_metadata', 'tileinfo_metadata', 'json_metadata']

items = query.item_collection()

for item in list(items):
    print(f"Examining scene: {item.id}")
    print(f"Available assets: {item.assets.keys()}")

    for band in bands_of_interest:
        if band in item.assets:
            print(f"Band: {band}")
            asset = item.assets[band]
            print(asset.href)

### Visualize STAC Results
We can optionally choose to visualize the Sentinel-2 scenes returned by our query. Code snippets from: https://odc-stac.readthedocs.io/en/latest/notebooks/stac-load-e84-aws.html

In this case, we can see that the scenes returned are from three separate tiles that overlap slightly. It is also important to remember that the STAC query has returned any scene that overlaps even slightly with our AOI (displayed on the map in red below), so we are transferring full Sentinel-2 scenes to our bucket, and we will do the clipping to the AOI boundary during the processing step later on. 

In [9]:
import folium
import shapely.geometry
from branca.element import Figure
import geopandas as gpd
import json  # For reading the GeoJSON file

# Load your existing data
gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")

# Compute granule id from components
gdf["granule"] = (
    gdf["mgrs:utm_zone"].apply(lambda x: f"{x:02d}")
    + gdf["mgrs:latitude_band"]
    + gdf["mgrs:grid_square"]
)

# Load the AOI GeoJSON file
with open('aois/Kakadu_National_Park.geojson', 'r') as f:
    aoi_geojson = json.load(f)

def convert_bounds(bbox, invert_y=False):
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

fig = Figure(width="400px", height="500px")
map1 = folium.Map()
fig.add_child(map1)

# Add the AOI GeoJSON to the map
folium.GeoJson(
    aoi_geojson,
    name="AOI Boundary",
    style_function=lambda x: dict(
        fill=True,
        fillColor='red',
        fillOpacity=0.2,
        weight=2,
        opacity=1,
        color='red'
    ),
).add_to(map1)

# Add your STAC data exploration layer
gdf.explore(
    "granule",
    categorical=True,
    tooltip=[
        "granule",
        "datetime",
        "eo:cloud_cover",
    ],
    popup=True,
    style_kwds=dict(fillOpacity=0.1, width=2),
    name="Scenes Returned ",
    m=map1,
)

map1.fit_bounds(bounds=convert_bounds(gdf.unary_union.bounds))

# Add layer control to toggle between layers
folium.LayerControl().add_to(map1)

display(fig)

/var/folders/fx/sjjzl8vn6nx36srv2y46mn_m0000gn/T/ipykernel_20217/2378216881.py:60: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  map1.fit_bounds(bounds=convert_bounds(gdf.unary_union.bounds))
/opt/homebrew/anaconda3/envs/aws-stac/lib/python3.13/site-packages/shapely/set_operations.py:421: RuntimeWarning: invalid value encountered in unary_union
  return lib.unary_union(collections, **kwargs)


## Transfer to S3 Bucket

To access the data, we can transfer the data from the source S3 bucket to our own S3 bucket. I have set up a bucket for testing called 'cs-awsopendata-sentinel2'.

In [3]:
import boto3
from urllib.parse import urlparse

def s3_to_s3_copy(source_url, destination_bucket='cs-awsopendata-sentinel2', prefix=''):
    # Parse the source URL to extract bucket and key.
    parsed_url = urlparse(source_url)
    
    # The source_url is in this format: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/52/L/HL/2023/1/S2B_52LHL_20230106_0_L2A/B02.tif
    url_parts = parsed_url.netloc.split('.') 
    
    # Source bucket should be 'sentinel-cogs' - the first part source url
    source_bucket = url_parts[0]
    
    # Source key is everything after the slash - e.g. 'sentinel-s2-l2a-cogs/52/L/HL/2023/1/S2B_52LHL_20230106_0_L2A/B02.tif'
    source_key = parsed_url.path.lstrip('/')

    # Construct destination key to match the source key. NOTE - can adjust destination_key as necessary.
    destination_key = f"{prefix}{source_key}"
    
    print(f"Copying from bucket: {source_bucket}, key: {source_key}")
    print(f"To bucket: {destination_bucket}, key: {destination_key}")
    
    # Initialize S3 client (region of the sentinel-cog bucket is us-west-2)
    s3 = boto3.client('s3', region_name='us-west-2')  # Specify region if needed
    
    try:
        response = s3.copy_object(
            CopySource={'Bucket': source_bucket, 'Key': source_key},
            Bucket=destination_bucket,
            Key=destination_key,
            RequestPayer='requester'
        )
        print(f"Copy successful: {response}")
        return destination_key
    
    except Exception as e:
        print(f"Error copying object: {e}")
        return None

In order to be able to properly process the files later and pass on information to end users, we also need to copy the metadata (included in the response to the STAC query) to our S3 bucket. The function below copies the metadata information for each 'item' (i.e. for each unique Sentinel-2 scene), which includes metadata properties for each band, such as the scale factor and offset values to apply to the rasters before they can be used for analysis. 

In [5]:
# TODO: should this be saved locally? be stored in memory and copied directly to S3? 
def export_metadata_json(item, out_path):
    with open(out_path, 'w') as f:
        item_dict = item.to_dict()
        json.dump(item_dict, f, indent=2)
        print(f"Metadata JSON saved to {out_path}")
    return

Run the function to copy the data returned from our previous query into our S3 bucket. 

In [6]:
# These are the bands that require scale factors applied to them.
bands_of_interest = ['aot', 'blue', 'coastal', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 
                     'rededge2', 'rededge3', 'swir16', 'swir22', 'wvp', 'scl', 'visual']

destination_bucket = 'cs-awsopendata-sentinel2'
prefix = 'kakadu-three-scenes/'


for item in query.item_collection():
    scene_id = item.id
    print(f"Processing scene: {scene_id}")
    
    metadata_json_path = f"{scene_id}_metadata.json"
    export_metadata_json(item, metadata_json_path)
    
    for band in bands_of_interest:
        if band in item.assets:
            asset = item.assets[band]
            source_url = asset.href
            
            print(f"Copying {band} band from {source_url}")
            destination_key = s3_to_s3_copy(source_url, destination_bucket, prefix)
            
            if destination_key:
                print(f"Successfully copied to s3://{destination_bucket}/{destination_key}")
            else:
                print(f"Failed to copy {band} band from {scene_id}")

Processing scene: S2A_52LHM_20230918_0_L2A
Metadata JSON saved to S2A_52LHM_20230918_0_L2A_metadata.json
Copying aot band from https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/52/L/HM/2023/9/S2A_52LHM_20230918_0_L2A/AOT.tif
Copying from bucket: sentinel-cogs, key: sentinel-s2-l2a-cogs/52/L/HM/2023/9/S2A_52LHM_20230918_0_L2A/AOT.tif
To bucket: cs-awsopendata-sentinel2, key: kakadu-three-scenes/sentinel-s2-l2a-cogs/52/L/HM/2023/9/S2A_52LHM_20230918_0_L2A/AOT.tif
Copy successful: {'ResponseMetadata': {'RequestId': 'WSGTM824G79SQ6JW', 'HostId': 'J7uP2RDtBgEi++m8QFSL+fdF+j0fpD9gSKFny8liUwyuJAO/GE8niZ3K4CPCyF3dmf09icau/Hw=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'J7uP2RDtBgEi++m8QFSL+fdF+j0fpD9gSKFny8liUwyuJAO/GE8niZ3K4CPCyF3dmf09icau/Hw=', 'x-amz-request-id': 'WSGTM824G79SQ6JW', 'date': 'Mon, 19 May 2025 07:14:19 GMT', 'x-amz-server-side-encryption': 'AES256', 'content-type': 'application/xml', 'content-length': '275', 'server': 'AmazonS3'}, 'RetryAttempt

## Download and visualize the raw data

First we'll download the tifs that are now stored in the cecil science S3 bucket and save them locally for testing processing.

In [ ]:
import boto3
import numpy as np
from pathlib import Path
import os
import re

def download_all_sentinel_bands(bucket_name, prefix, local_base_dir="./sentinel_data"):

    # Initialize S3 client
    s3 = boto3.client('s3')
    
    print(f"Listing objects in bucket '{bucket_name}' with prefix '{prefix}'...")
    
    # Dictionary to store scene IDs and their downloaded files
    downloaded_files = {}
    
    # Get all objects in the specified subdirectory ('prefix')
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, MaxKeys=100)
    
    for obj in response['Contents']:
        key = obj['Key']
        
        # Extract scene ID from the path (pattern: .../sentinel-s2-l2a-cogs/52/L/HM/2023/9/S2A_52LHM_20230918_0_L2A/B02.tif)
        match = re.search(r'/(S2[AB]_\d+[A-Z]+_\d+_\d+_L\d+[A-Z])/', key)
        if not match:
            print(f"Could not extract scene ID from path: {key}")
            continue
        # Extract scene ID and filename.
        scene_id = match.group(1)
        filename = os.path.basename(key)
        
        # Create directory for this scene.
        scene_dir = os.path.join(local_base_dir, scene_id)
        Path(scene_dir).mkdir(parents=True, exist_ok=True)
        
        # Download the file
        local_path = os.path.join(scene_dir, filename)
        
        print(f"Downloading {key} to {local_path}")
        s3.download_file(bucket_name, key, local_path)
        
        # Update the downloaded files dictionary
        if scene_id not in downloaded_files:
            downloaded_files[scene_id] = []
        downloaded_files[scene_id].append(local_path)
    
    # Print summary
    print("\nDownload Summary:")
    for scene_id, files in downloaded_files.items():
        print(f"  Scene: {scene_id}")
        print(f"    - Downloaded {len(files)} files")
    
    return downloaded_files

In [ ]:
    # Bucket and subdirectory within bucket from which to download data.
    bucket_name = "cs-awsopendata-sentinel2"
    prefix = "kakadu-rgb-test/"
    
    # Download the files for all the bands in the 'prefix' folder.
    print(f"Downloading all band files from {bucket_name}/{prefix}...")
    downloaded_scenes = download_all_sentinel_bands(
        bucket_name=bucket_name,
        prefix=prefix,
        local_base_dir="./sentinel_data" # Specify where to save the files locally. 
    )

In [ ]:
import rasterio
import matplotlib.pyplot as plt

for scene_id, files in downloaded_scenes.items():
    print(f"Scene ID: {scene_id}")

    # Set up three subplots for the three bands
    fig, axs = plt.subplots(1, len(files), figsize=(15, 5))

    for i in range(len(files)):
        with rasterio.open(files[i]) as src:
            print(f"  - CRS: {src.crs}")
            print(f"  - Width: {src.width}, Height: {src.height}")
            print(f"  - Bounds: {src.bounds}")

            axs[i].set_title(f"{files[i]}")
            axs[i].imshow(src.read(1), cmap='gray')


## Examine Scale and Offset Metadata

In [ ]:
# import rioxarray

# test_band = rioxarray.open_rasterio('https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/52/L/HM/2023/9/S2A_52LHM_20230918_0_L2A/B02.tif')
# print(test_band)

## Process the data into a DataFrame

In [ ]:
import rasterio.features
import rasterio.mask
import pandas as pd
import numpy as np

def create_geom_mask(src, geom):
    return rasterio.features.geometry_mask(
        geometries=[geom],
        transform=src.transform,
        all_touched=True,
        out_shape=(src.height, src.width),
    )

def load_geojson_as_geom(geojson_path, target_crs=None):
    gdf = gpd.read_file(geojson_path)
    if target_crs:
        gdf = gdf.to_crs(target_crs)
    return gdf.geometry.iloc[0]

def get_pixel_boundary_wkt(dataset, row, col):
    points = [
        dataset.xy(row, col, offset=offset) for offset in ["ul", "ll", "lr", "ur", "ul"]
    ]

    return "POLYGON ((" + ", ".join(f"{x} {y}" for x, y in points) + "))"

def process_s2(dataset, geojson_path):
    """Original process() function from parse_geotiff.py - just removed
    position, which wasn't used for local testing."""
    
    crs = str(dataset.crs)

    geom = load_geojson_as_geom(geojson_path, target_crs=crs)

    geom_mask = create_geom_mask(dataset, geom)
    
    results = []

    for band_index in range(1, dataset.count + 1):

        raster_band = dataset.read(band_index, masked=True)

        for raster_row in range(0, dataset.height):
            for raster_col in range(0, dataset.width):

                # Skip pixels outside the geometry
                if geom_mask[raster_row][raster_col]:
                    continue

                if raster_band.mask[raster_row][raster_col]:
                    value = None
                else:
                    value = float(raster_band.data[raster_row][raster_col])

                x, y = dataset.xy(raster_row, raster_col, offset="center")
                
                pixel_boundary = get_pixel_boundary_wkt(dataset, raster_row, raster_col)
                
                results.append({
                    'relative_path': dataset.name.split('/')[-2:],
                    'crs': crs,
                    'x': x,
                    'y': y,
                    'boundary_wkt': pixel_boundary,
                    'band_index': band_index,
                    'value': value
                })
    
    return pd.DataFrame(results)

In [ ]:
s2_band = "/Users/sabinenix/Documents/Code/sentinel-2-pipeline/aws/sentinel_data/S2A_52LHM_20230918_0_L2A/B02.tif"
geojson_path = "/Users/sabinenix/Documents/Data/AOIs/Kakadu_National_Park.geojson"

with rasterio.open(s2_band) as src:
    s2_band_results = process_s2(src, geojson_path)

In [ ]:
s2_band_results.head(5)

In [ ]:
from shapely import wkt
import geopandas as gpd
import matplotlib.pyplot as plt

# First create your base plot with the s2 data
fig, ax = plt.subplots(figsize=(10, 10))
s2_gdf = gpd.GeoDataFrame(s2_band_results, 
                         geometry=gpd.GeoSeries.from_wkt(s2_band_results['boundary_wkt']), 
                         crs=s2_band_results.iloc[0]['crs'])
s2_gdf.plot(column='value', cmap='viridis', legend=True, ax=ax)

# Now load and overlay the GeoJSON boundary
# Replace 'path_to_your_geojson.geojson' with your actual file path
aoi_gdf = gpd.read_file(geojson_path)

# If needed, reproject to match the CRS of your s2_gdf
if aoi_gdf.crs != s2_gdf.crs:
    aoi_gdf = aoi_gdf.to_crs(s2_gdf.crs)

# Plot the AOI boundary on top of the existing plot with a distinct style
aoi_gdf.boundary.plot(ax=ax, color='red', linewidth=2)

# Optional: Add a title and labels
plt.title('S2 Data with AOI Boundary')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.tight_layout()
plt.show()